In [1]:
# loading the data
import pandas as pd
data = pd.read_csv("data/orders_and_shipments.csv")
data.head(3)

,Order ID,Order Item ID,Order YearMonth,Order Year,Order Month,Order Day,Order Time,Order Quantity,Product Department,Product Category,Product Name,Customer ID,Customer Market,Customer Region,Customer Country,Warehouse Country,Shipment Year,Shipment Month,Shipment Day,Shipment Mode,Shipment Days - Scheduled,Gross Sales,Discount %,Profit
0,3535,8793,201502,2015,2,21,14:07,1,Fan Shop,Fishing,Field & Stream Sportsman 16 Gun Fire Safe,7840,LATAM,Central America,Mexico,Puerto Rico,2015,2,27,Standard Class,4,400,0.25,200
1,4133,10320,201503,2015,3,2,07:37,1,Fan Shop,Fishing,Field & Stream Sportsman 16 Gun Fire Safe,1203,LATAM,South America,Brazil,Puerto Rico,2015,3,6,Standard Class,4,400,0.09,200
2,7396,18517,201504,2015,4,18,22:47,1,Fan Shop,Fishing,Field & Stream Sportsman 16 Gun Fire Safe,10866,LATAM,Central America,Mexico,Puerto Rico,2015,4,20,Standard Class,4,400,0.06,200


In [2]:
# check the column names
print(data.columns)

Index(['Order ID ', ' Order Item ID ', ' Order YearMonth ', ' Order Year ',
       ' Order Month ', ' Order Day ', 'Order Time', 'Order Quantity',
       'Product Department', 'Product Category', 'Product Name',
       ' Customer ID ', 'Customer Market', 'Customer Region',
       'Customer Country', 'Warehouse Country', 'Shipment Year',
       'Shipment Month', 'Shipment Day', 'Shipment Mode',
       ' Shipment Days - Scheduled ', ' Gross Sales ', ' Discount % ',
       ' Profit '],
      dtype='object')


In [5]:
# remove extra spaces from column names
data.columns = data.columns.str.strip()
print(data.columns)

Index(['Order ID', 'Order Item ID', 'Order YearMonth', 'Order Year',
       'Order Month', 'Order Day', 'Order Time', 'Order Quantity',
       'Product Department', 'Product Category', 'Product Name', 'Customer ID',
       'Customer Market', 'Customer Region', 'Customer Country',
       'Warehouse Country', 'Shipment Year', 'Shipment Month', 'Shipment Day',
       'Shipment Mode', 'Shipment Days - Scheduled', 'Gross Sales',
       'Discount %', 'Profit'],
      dtype='object')


In [3]:
def create_date_columns(df, day_col, month_col, year_col, col_name):
    """
    Create a new column with a datetime object from three columns with day, month and year.
    """
    df[col_name] = pd.to_datetime(df[month_col].astype(str) + '-' + df[day_col].astype(str) + '-' + df[year_col].astype(str))

In [6]:
# create order date column
create_date_columns(data, 'Order Day', 'Order Month', 'Order Year', 'Order Date')

# create shipment date column
create_date_columns(data, 'Shipment Day', 'Shipment Month', 'Shipment Year', 'Shipment Date')

In [7]:
def create_datetime_columns(df, day_col, month_col, year_col, time_col, col_name):
    """
    Create a new column with a datetime object from three columns with day, month and year.
    """
    df[col_name] = pd.to_datetime(df[month_col].astype(str) + '-' + df[day_col].astype(str) + '-' + df[year_col].astype(str) + ' ' + df[time_col].astype(str))

In [8]:
# create order datetime column
create_datetime_columns(data, 'Order Day', 'Order Month', 'Order Year', 'Order Time', 'Order Datetime')

In [9]:
# create shipment days column [actual time to place shipment]
data['Shipment Days'] = (data['Shipment Date'] - data['Order Date']).dt.days

In [10]:
import numpy as np

# create shipment delay column
data['Shipment Delay'] = np.where(data['Shipment Days - Scheduled'] > data['Shipment Days'], 0, data['Shipment Days'] - data['Shipment Days - Scheduled'])

In [11]:
# if shipment delay is negative there might be error in the shipment month
fil_df = np.where(data['Shipment Days'] < 0 )

In [12]:
filtered_shipment_date = data.loc[fil_df, 'Shipment Date']
filtered_order_date = data.loc[fil_df, 'Order Date']
filtered_shipment_day = data.loc[fil_df, 'Shipment Day']
filtered_shipment_month = data.loc[fil_df, 'Shipment Month']
filtered_shipment_year = data.loc[fil_df, 'Shipment Year']
filtered_order_day = data.loc[fil_df, 'Order Day']
filtered_order_month = data.loc[fil_df, 'Order Month']
filtered_order_year = data.loc[fil_df, 'Order Year']

# Create a new DataFrame to combine the 'Shipment Date' and 'Order Date'
combined_data = pd.DataFrame({'Order Day': filtered_order_day, 'Order Month': filtered_order_month, 'Order Year': filtered_order_year, 
                              'Order Date': filtered_order_date, 
                              'Shipment Day': filtered_shipment_day, 'Shipment Month': filtered_shipment_month, 'Shipment Year': filtered_shipment_year,
                              'Shipment Date': filtered_shipment_date, })

# Print the combined DataFrame
combined_data

,Order Day,Order Month,Order Year,Order Date,Shipment Day,Shipment Month,Shipment Year,Shipment Date
294,10,2,2015,2015-02-10,20,1,2015,2015-01-20
295,2,3,2015,2015-03-02,14,2,2015,2015-02-14
296,4,4,2015,2015-04-04,17,3,2015,2015-03-17
297,6,4,2015,2015-04-06,23,3,2015,2015-03-23
298,18,9,2015,2015-09-18,19,7,2015,2015-07-19
...,...,...,...,...,...,...,...,...
30560,25,5,2017,2017-05-25,16,6,2016,2016-06-16
30561,13,7,2017,2017-07-13,12,7,2016,2016-07-12
30562,21,7,2017,2017-07-21,13,7,2016,2016-07-13
30563,10,8,2017,2017-08-10,21,7,2016,2016-07-21


So there are 2735 orders which shipment dates preceed the order dates which is not possible in a real life scenario. So, I have to remove those from data.

In [13]:
wrong_indices = np.where(data['Shipment Date'] < data['Order Date'])
data.loc[wrong_indices, 'Shipment Date'].count()

# Remove the wrong rows from the DataFrame
mask = np.where(data['Shipment Date'] >= data['Order Date'])
data = data.loc[mask]

In [14]:
# check the customer country column values
data['Customer Country'].unique()

array(['Mexico', 'Brazil', 'Denmark', 'Netherlands', 'Germany', 'China',
       'Indonesia', 'Pakistan', 'India', 'USA', 'Hungary', 'Sudan',
       'Democratic Republic of Congo', 'Poland', 'Togo', 'Guatemala',
       'Panama', 'Chile', 'France', 'Sweden', 'Dominican�Republic',
       'Venezuela', 'South Korea', 'Madagascar', 'Iran', 'Cuba',
       'Nicaragua', 'United Kingdom', 'Afghanistan', 'Singapore',
       'Morocco', 'Spain', 'Niger', 'Turkey', 'South Africa', 'Iraq',
       'Honduras', 'Italy', 'Australia', 'Cote d�Ivoire', 'Croatia',
       'Ecuador', 'Syria', 'Haiti', 'Bangladesh', 'Argentina', 'Romania',
       'El Salvador', 'Vietnam', 'Japan', 'Nigeria', 'Belarus',
       'Uzbekistan', 'Egypt', 'Albania', 'Georgia', 'Cameroon',
       'Colombia', 'New zealand', 'Thailand', 'Senegal', 'Per�',
       'Ukraine', 'Canada', 'Belgium', 'Philippines', 'Austria',
       'Uruguay', 'Malaysia', 'Hong Kong', 'Algeria�', 'Saudi Arabia',
       'Russia', 'Switzerland', 'Ireland', 'Bulg

In [15]:
data['Customer Country'] = np.where(data['Customer Country'] == 'Cote d�Ivoire', "Cote d'Ivoire", data['Customer Country'])
data['Customer Country'] = np.where(data['Customer Country'] == 'Dominican�Republic', 'Dominican Republic', data['Customer Country'])
data['Customer Country'] = np.where(data['Customer Country'] == 'Per�', 'Peru', data['Customer Country'])
data['Customer Country'] = np.where(data['Customer Country'] == 'Algeria�', 'Algeria', data['Customer Country'])
data['Customer Country'] = np.where(data['Customer Country'] == 'Israel�', 'Israel', data['Customer Country'])
data['Customer Country'] = np.where(data['Customer Country'] == 'Ben�n', 'Benin', data['Customer Country'])

In [16]:
data['Discount %'] = pd.to_numeric(data['Discount %'], errors='coerce')

# Calculate the discounted price for each row
data['Net Sales'] = data['Gross Sales'] - (data['Gross Sales'] * (data['Discount %'] / 100))

In [17]:
data.to_csv('data/orders_and_shipments_transformed.csv', index=False)

## ✍️ Judging criteria
| CATEGORY | WEIGHTING | DETAILS                                                              |
|:---------|:----------|:---------------------------------------------------------------------|
| **Visualizations** | 35% | <ul><li>Appropriateness of visualizations used.</li><li>Clarity of insight from visualizations.</li></ul> |
| **Insights** | 25%       | <ul><li>Clarity of insights - how clear and well presented the insights are.</li><li>Quality of recommendations - are appropriate analytical techniques used & are the conclusions valid?</li><li>Number of relevant insights found for the target audience.</li></ul>       |
| **Storytelling**  | 25%       | <ul><li>How well the data and insights are connected to tell a story.</li><li>How the narrative and whole report connects together.</li><li>How balanced the report is: in-depth enough but also concise.</li></ul> |
| **Votes** | 15% | <ul><li>Up voting - most upvoted entries get the most points.</li></ul> |

## 🧾 Executive summary
The occurrence of shipment delays escalated gradually over time, reaching its highest point in November 2017, but significantly reduced in December 2017. This change in shipment delays attributed to the fact that orders ceased in March 2017. Consequently, the majority of pending orders were fulfilled during November 2017, leading to a substantial increase in shipment delays during that month. However, once these backlog orders were delivered, the shipment delays decreased significantly in December 2017.  
Footwear products experience the highest average shipment delay days.  
The occurrence of overstock phenomena surpasses that of understock phenomena.

## 📷 Dashboard screenshot
![image.png](images/1.png)  
![image.png](images/2.png)  
![image.png](images/3.png)

## 🌐 Upload your dashboard
https://public.tableau.com/views/SupplyChainAnalytics_16895737251610/ShipmentDelayAnalysis?:language=en-US&:display_count=n&:origin=viz_share_link